<a href="https://colab.research.google.com/github/Analeidi/AI-Capstone/blob/main/readingData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reading Data

In [1]:
import sqlite3
import json
from datetime import datetime

In [2]:
timeframe = '2009-01'
sql_transaction = []

connection = sqlite3.connect("/content/drive/MyDrive/Colab Notebooks/'{}.db'.format(timeframe)")
c = connection.cursor()

In [3]:
def create_table():
    c.execute("""CREATE TABLE IF NOT EXISTS parent_reply 
    (parent_id TEXT PRIMARY KEY, comment_id TEXT UNIQUE, parent TEXT, 
    comment TEXT, subreddit TEXT, unix INT, score INT)""")


def format_data(data):
    data = data.replace("\n", "  newlinechar  ").replace("\r", "  newlinechar  ").replace(' " ', " ' ")
    return data


def find_existing_score(pid):
    try: 
        sql = "SELECT score FROM parent_reply WHERE parent_id = '{}' LIMIT 1".format(pid)
        c.execute(sql)
        result = c.fetchone()
        if result != None:
            return result[0]
        else:
            return False
    except Exception as e:
        print("find_parent", e)
        return False

In [4]:
def acceptable(data):
    if len(data.split(' ')) > 50 or len(data) < 1:
        return False
    elif len(data) > 10000:
        return False
    elif data == '[deleted]' or data == '[removed]':
        return False
    else:
        return True


def find_parent(pid):
    try: 
        sql = "SELECT comment FROM parent_reply WHERE comment_id = '{}' LIMIT 1".format(pid)
        c.execute(sql)
        result = c.fetchone()
        if result != None:
            return result[0]
        else:
            return False
    except Exception as e:
        print("find_parent", e)
        return False
        
def transaction_bldr(sql):
    global sql_transaction
    sql_transaction.append(sql)
    if len(sql_transaction) > 1000:
        c.execute('BEGIN TRANSACTION')
        for s in sql_transaction:
            try:
                c.execute(s)
            except:
                pass
        connection.commit()
        sql_transaction = []


In [5]:
def sql_insert_replace_comment(commentid,parentid,parent,comment,subreddit,time,score):
    try:
        sql = """UPDATE parent_reply SET parent_id = ?, comment_id = ?, parent = ?, comment = ?, subreddit = ?, unix = ?, score = ? WHERE parent_id =?;""".format(parentid, commentid, parent, comment, subreddit, int(time), score, parentid)
        transaction_bldr(sql)
    except Exception as e:
        print('s0 insertion',str(e))


def sql_insert_has_parent(commentid,parentid,parent,comment,subreddit,time,score):
    try:
        sql = """INSERT INTO parent_reply (parent_id, comment_id, parent, comment, subreddit, unix, score) VALUES ("{}","{}","{}","{}","{}",{},{});""".format(parentid, commentid, parent, comment, subreddit, int(time), score)
        transaction_bldr(sql)
    except Exception as e:
        print('s0 insertion',str(e))


def sql_insert_no_parent(commentid,parentid,comment,subreddit,time,score):
    try:
        sql = """INSERT INTO parent_reply (parent_id, comment_id, comment, subreddit, unix, score) VALUES ("{}","{}","{}","{}",{},{});""".format(parentid, commentid, comment, subreddit, int(time), score)
        transaction_bldr(sql)
    except Exception as e:
        print('s0 insertion',str(e))

In [9]:
if __name__== "__main__":
    create_table()
    row_counter = 0 
    row_counter = 0 
    paired_rows = 0 
    #The data below is being encoded with latin-1 not utf-8
    with open("/content/drive/MyDrive/Colab Notebooks/RC_2009-01".format(timeframe.split('-')[0], timeframe), buffering=1000) as f:
        for row in f:
            row_counter += 1
            row = json.loads(row)
            parent_id = row['parent_id']
            body = format_data(row['body'])
            created_utc = row['created_utc']
            score = row['score']
            subreddit = row['subreddit']
            parent_data = find_parent(parent_id) 
            comment_id = row['name']

            if row_counter > 1000000:
                break
            if score >= 2:
                if acceptable(body):
                    existing_comment_score = find_existing_score(parent_id)
                    if existing_comment_score:
                        if score > existing_comment_score:
                            sql_insert_replace_comment(comment_id, parent_id, parent_data, body, subreddit, created_utc, score)
                    else:
                        if parent_data:
                            sql_insert_has_parent(comment_id, parent_id, parent_data, body, subreddit, created_utc, score)
                            paired_rows += 1
                        else:
                            sql_insert_no_parent(comment_id, parent_id, body, subreddit, created_utc, score)

            if row_counter % 10000 == 0:
                print("Total rows read: {}, Paired rows: {}, Time: {}".format(row_counter, paired_rows, str(datetime.now())))

Total rows read: 10000, Paired rows: 20, Time: 2020-12-10 09:36:53.758756
Total rows read: 20000, Paired rows: 49, Time: 2020-12-10 09:36:54.235226
Total rows read: 30000, Paired rows: 71, Time: 2020-12-10 09:36:54.735013
Total rows read: 40000, Paired rows: 104, Time: 2020-12-10 09:36:55.217234
Total rows read: 50000, Paired rows: 139, Time: 2020-12-10 09:36:55.699212
Total rows read: 60000, Paired rows: 159, Time: 2020-12-10 09:36:56.188746
Total rows read: 70000, Paired rows: 182, Time: 2020-12-10 09:36:56.704922
Total rows read: 80000, Paired rows: 201, Time: 2020-12-10 09:36:57.203596
Total rows read: 90000, Paired rows: 233, Time: 2020-12-10 09:36:57.722054
Total rows read: 100000, Paired rows: 247, Time: 2020-12-10 09:36:58.207171
Total rows read: 110000, Paired rows: 269, Time: 2020-12-10 09:36:58.677903
Total rows read: 120000, Paired rows: 287, Time: 2020-12-10 09:36:59.174845
Total rows read: 130000, Paired rows: 310, Time: 2020-12-10 09:36:59.649864
Total rows read: 140000,